In [2]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
sys.path.insert(1, p)
sys.path.insert(1, pp)

In [38]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
import config.config as conf
import networkx as nx

In [4]:
# 파일 불러오기
with open(f'{conf.DATA_PATH}/data/tag_net_py.pkl', 'rb') as f:
    tag_net_py = pickle.load(f)


In [5]:
# 파일 불러오기
with open(f'{conf.DATA_PATH}/data/tag_net_java.pkl', 'rb') as f:
    tag_net_java = pickle.load(f)


In [7]:
tag_net_py['creationdate']      = pd.to_datetime(tag_net_py['creationdate'])
tag_net_java['creationdate']    = pd.to_datetime(tag_net_java['creationdate'])

In [8]:
tag_net_java.head()

,id,creationdate,tags
0,70162777,2021-11-30 00:08:54.470,<java><android><asynchronous><arraylist><andro...
1,70162790,2021-11-30 00:11:20.733,<java><excel>
2,70162808,2021-11-30 00:14:26.370,<java><jvm>
3,70162810,2021-11-30 00:14:49.373,<java><spring-boot><resttemplate><mockrestserv...
4,70162880,2021-11-30 00:26:16.910,<java><json><date><jsonschema><jsonschema2pojo>


In [18]:
tag_net_py['taglist'] = tag_net_py['tags'].apply(lambda x: [y for y in x.replace('><', ' ').replace('<', '').replace('>', '').strip().split(' ') if y !='python'])
tag_net_java['taglist'] = tag_net_java['tags'].apply(lambda x: [y for y in x.replace('><', ' ').replace('<', '').replace('>', '').strip().split(' ') if y !='java'])

In [19]:
tag_net_py.head()

,id,creationdate,tags,taglist
0,70162722,2021-11-30 00:00:11.787,<python><python-3.x><unit-testing>,"[python-3.x, unit-testing]"
1,70162741,2021-11-30 00:03:41.067,<python><pandas><dataframe><group-by><data-ana...,"[pandas, dataframe, group-by, data-analysis]"
2,70162770,2021-11-30 00:07:54.443,<python><string><find><getvalue>,"[string, find, getvalue]"
3,70162816,2021-11-30 00:15:44.530,<python><pandas><csv>,"[pandas, csv]"
4,70162820,2021-11-30 00:16:44.513,<python><pyspark><window-functions>,"[pyspark, window-functions]"


In [21]:
py_tag_df = tag_net_py.apply(pd.Series.explode)[['id', 'creationdate', 'taglist']]

In [22]:
java_tag_df = tag_net_java.apply(pd.Series.explode)[['id', 'creationdate', 'taglist']]

In [23]:
java_tag_df.head()

,id,creationdate,taglist
0,70162777,2021-11-30 00:08:54.470,android
0,70162777,2021-11-30 00:08:54.470,asynchronous
0,70162777,2021-11-30 00:08:54.470,arraylist
0,70162777,2021-11-30 00:08:54.470,android-volley
1,70162790,2021-11-30 00:11:20.733,excel


In [24]:
java_tag_df['core_tag'] = 'java'
py_tag_df['core_tag'] = 'python'

In [ ]:
java_tag_df[['core_tag', 'taglist']]

,id,creationdate,taglist,core_tag
0,70162777,2021-11-30 00:08:54.470,android,java
0,70162777,2021-11-30 00:08:54.470,asynchronous,java
0,70162777,2021-11-30 00:08:54.470,arraylist,java
0,70162777,2021-11-30 00:08:54.470,android-volley,java
1,70162790,2021-11-30 00:11:20.733,excel,java
...,...,...,...,...
88119,77574984,2023-11-29 23:12:54.717,binary-search-tree,java
88120,77575008,2023-11-29 23:18:36.953,jdbc,java
88120,77575008,2023-11-29 23:18:36.953,mariadb,java
88121,77575116,2023-11-29 23:50:29.087,jpa,java


In [29]:
tag_df = pd.concat([py_tag_df[['core_tag', 'taglist']], java_tag_df[['core_tag', 'taglist']]], axis =0).reset_index(drop=True)

In [30]:
tag_df.head()

,core_tag,taglist
0,python,python-3.x
1,python,unit-testing
2,python,pandas
3,python,dataframe
4,python,group-by


In [31]:
tag_df.columns = ['Source', 'Target']

In [34]:
tag_df.dropna(inplace=True)

In [35]:
node = np.unique(np.append(tag_df['Source'].values, tag_df['Target'].values))

In [36]:
# 질문자->답변자 로 edgelist 생성
edgelist = list(zip(tag_df['Source'], tag_df['Target']))

In [39]:
tag_df.to_csv('./gephi.csv', index=False)